# BREDA+极小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_robust1=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.1
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")
neighbor_robust1_help=[]
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust1=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_robust1*100)):
    x_average_resource_quantity_robust1[i][0]=agent_resource_quantity_robust1[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    x_average_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
'''sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+(1/(100-int(proportion_robust1*100)))*(-beta_robust1[i]*alpha_robust1*math.log(1+agent_resource_quantity_robust1[i][0]))
average_cost_honest_robust1[0][0]=sum_robust1'''

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust1_help=0\n    for j in range(0,15):\n        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:\n            q_robust1_help=q_robust1_help+1\n    q_robust1.append(q_robust1_help) '

In [4]:
def BREDA_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,x_average_resource_quantity_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA+极小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
        
        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_robust1*100)):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]+iteration_update_robust1*((x_average_resource_quantity_robust1[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]-iteration_update_robust1*(((2*a_robust1[i]*agent_resource_quantity_robust1[i][k-1]-2*a_robust1[i]*b_robust1[i])/100)+(dual_variable_robust1[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_robust1*100)),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]
                
        #TM鲁棒聚合 每个用户聚合自身以及邻居的动态一致平均值  诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_robust1*100)):
            sort_list_robust1=[]
            sum_x_robust1=0
            for j in range (0,15):
                sort_list_robust1.append(x_average_resource_quantity_robust1[neighbor_robust1[i,j]][k-1])
            sort_list_robust1.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust1)):
                del sort_list_robust1[14-j]
            for j in range(0,int(q_robust1)):
                del sort_list_robust1[0]     
            for j in range(0,len(sort_list_robust1)):
                sum_x_robust1=sum_x_robust1+sort_list_robust1[j]
            
            x_average_resource_quantity_robust1[i][k]=(1/(16-2*q_robust1))*(x_average_resource_quantity_robust1[i][k-1]+sum_x_robust1)+(agent_resource_quantity_robust1[i][k]-agent_resource_quantity_robust1[i][k-1])      
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_robust1*100)),100):
            x_average_resource_quantity_robust1[i][k]=x_average_resource_quantity_robust1[i][k-1]  
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust1[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        #iteration_update_robust1=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
BREDA_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,x_average_resource_quantity_robust1)

****************************BREDA+极小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.982054894276
agent 2 在 3999 次迭代中的资源申请量为: 57.390265420762425
agent 3 在 3999 次迭代中的资源申请量为: 62.224537583189544
agent 4 在 3999 次迭代中的资源申请量为: 39.58463784186244
agent 5 在 3999 次迭代中的资源申请量为: 59.20182256793325
agent 6 在 3999 次迭代中的资源申请量为: 57.074743111601016
agent 7 在 3999 次迭代中的资源申请量为: 67.15853259229364
agent 8 在 3999 次迭代中的资源申请量为: 48.198072903263736
agent 9 在 3999 次迭代中的资源申请量为: 67.11492294693133
agent 10 在 3999 次迭代中的资源申请量为: 55.6055706004405
agent 11 在 3999 次迭代中的资源申请量为: 67.16365524666857
agent 12 在 3999 次迭代中的资源申请量为: 40.25793790242583
agent 13 在 3999 次迭代中的资源申请量为: 51.5120992416792
agent 14 在 3999 次迭代中的资源申请量为: 43.02005410926002
agent 15 在 3999 次迭代中的资源申请量为: 44.36563913798073
agent 16 在 3999 次迭代中的资源申请量为: 51.28383663398827
agent 17 在 3999 次迭代中的资源申请量为: 51.90030551974406
agent 18 在 3999 次迭代中的资源申请量为: 48.40937277649653
agent 19 在 3999 次迭代中的资源申请量为: 46.67437133234524
agent 20 在 3999 次迭代中的资源申请量为: 52.43263742932918
agent 21 在 3999 次迭代中的资源申请量为: 71.21967600182437
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.975809470599756
agent 2 在 5999 次迭代中的资源申请量为: 57.38232798035258
agent 3 在 5999 次迭代中的资源申请量为: 62.22149226779774
agent 4 在 5999 次迭代中的资源申请量为: 39.55893995790301
agent 5 在 5999 次迭代中的资源申请量为: 59.19611002101361
agent 6 在 5999 次迭代中的资源申请量为: 57.06686735752354
agent 7 在 5999 次迭代中的资源申请量为: 67.16118048220463
agent 8 在 5999 次迭代中的资源申请量为: 48.181668455190156
agent 9 在 5999 次迭代中的资源申请量为: 67.11651416610346
agent 10 在 5999 次迭代中的资源申请量为: 55.596999952607064
agent 11 在 5999 次迭代中的资源申请量为: 67.1678536509241
agent 12 在 5999 次迭代中的资源申请量为: 40.23252827733868
agent 13 在 5999 次迭代中的资源申请量为: 51.49984321945356
agent 14 在 5999 次迭代中的资源申请量为: 42.99761410655125
agent 15 在 5999 次迭代中的资源申请量为: 44.345129196158425
agent 16 在 5999 次迭代中的资源申请量为: 51.27121950007364
agent 17 在 5999 次迭代中的资源申请量为: 51.88750360438292
agent 18 在 5999 次迭代中的资源申请量为: 48.393207452837686
agent 19 在 5999 次迭代中的资源申请量为: 46.656992310746624
agent 20 在 5999 次迭代中的资源申请量为: 52.42018016639655
agent 21 在 5999 次迭代中的资源申请量为: 71.2265753927996
agen

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.96915447498345
agent 2 在 7999 次迭代中的资源申请量为: 57.37558731148277
agent 3 在 7999 次迭代中的资源申请量为: 62.21489668020368
agent 4 在 7999 次迭代中的资源申请量为: 39.55181919325981
agent 5 在 7999 次迭代中的资源申请量为: 59.18945569312952
agent 6 在 7999 次迭代中的资源申请量为: 57.0601445742699
agent 7 在 7999 次迭代中的资源申请量为: 67.15476909176486
agent 8 在 7999 次迭代中的资源申请量为: 48.174716726883425
agent 9 在 7999 次迭代中的资源申请量为: 67.11006038544562
agent 10 在 7999 次迭代中的资源申请量为: 55.590260631662055
agent 11 在 7999 次迭代中的资源申请量为: 67.1615233727629
agent 12 在 7999 次迭代中的资源申请量为: 40.22541287853681
agent 13 在 7999 次迭代中的资源申请量为: 51.49298274643115
agent 14 在 7999 次迭代中的资源申请量为: 42.99053545545632
agent 15 在 7999 次迭代中的资源申请量为: 44.338102049042156
agent 16 在 7999 次迭代中的资源申请量为: 51.264387139006644
agent 17 在 7999 次迭代中的资源申请量为: 51.88064790279126
agent 18 在 7999 次迭代中的资源申请量为: 48.38625794073771
agent 19 在 7999 次迭代中的资源申请量为: 46.65001088646328
agent 20 在 7999 次迭代中的资源申请量为: 52.41332483182273
agent 21 在 7999 次迭代中的资源申请量为: 71.22030030569793
agent 

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.96875131864894
agent 2 在 9999 次迭代中的资源申请量为: 57.37518022547809
agent 3 在 9999 次迭代中的资源申请量为: 62.214487589299615
agent 4 在 9999 次迭代中的资源申请量为: 39.55143807889142
agent 5 在 9999 次迭代中的资源申请量为: 59.189050305079974
agent 6 在 9999 次迭代中的资源申请量为: 57.05973971483532
agent 7 在 9999 次迭代中的资源申请量为: 67.15436168871946
agent 8 在 9999 次迭代中的资源申请量为: 48.174319297588106
agent 9 在 9999 次迭代中的资源申请量为: 67.10965179866697
agent 10 在 9999 次迭代中的资源申请量为: 55.58985633036076
agent 11 在 9999 次迭代中的资源申请量为: 67.16111985169302
agent 12 在 9999 次迭代中的资源申请量为: 40.22503117754027
agent 13 在 9999 次迭代中的资源申请量为: 51.49257903742341
agent 14 在 9999 次迭代中的资源申请量为: 42.99014684449475
agent 15 在 9999 次迭代中的资源申请量为: 44.337712133795016
agent 16 在 9999 次迭代中的资源申请量为: 51.26398895317381
agent 17 在 9999 次迭代中的资源申请量为: 51.880247378448175
agent 18 在 9999 次迭代中的资源申请量为: 48.38585977222243
agent 19 在 9999 次迭代中的资源申请量为: 46.64961373107119
agent 20 在 9999 次迭代中的资源申请量为: 52.4129228499336
agent 21 在 9999 次迭代中的资源申请量为: 71.21989532111685
agen

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.968747115850675
agent 2 在 51999 次迭代中的资源申请量为: 57.37517582550517
agent 3 在 51999 次迭代中的资源申请量为: 62.21448273460106
agent 4 在 51999 次迭代中的资源申请量为: 39.55143641630589
agent 5 在 51999 次迭代中的资源申请量为: 59.18904590023989
agent 6 在 51999 次迭代中的资源申请量为: 57.05973547603986
agent 7 在 51999 次迭代中的资源申请量为: 67.15435664802436
agent 8 在 51999 次迭代中的资源申请量为: 48.1743160725415
agent 9 在 51999 次迭代中的资源申请量为: 67.10964672927383
agent 10 在 51999 次迭代中的资源申请量为: 55.5898521809298
agent 11 在 51999 次迭代中的资源申请量为: 67.16111501592275
agent 12 在 51999 次迭代中的资源申请量为: 40.225029452920126
agent 13 在 51999 次迭代中的资源申请量为: 51.49257512903627
agent 14 在 51999 次迭代中的资源申请量为: 42.99014453177624
agent 15 在 51999 次迭代中的资源申请量为: 44.337709643029484
agent 16 在 51999 次迭代中的资源申请量为: 51.26398548166408
agent 17 在 51999 次迭代中的资源申请量为: 51.88024374435723
agent 18 在 51999 次迭代中的资源申请量为: 48.38585648075609
agent 19 在 51999 次迭代中的资源申请量为: 46.649610570897806
agent 20 在 51999 次迭代中的资源申请量为: 52.412919091258004
agent 21 在 51999 次迭代中的资源申请量为: 7

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.968747115850675
agent 2 在 53999 次迭代中的资源申请量为: 57.37517582550517
agent 3 在 53999 次迭代中的资源申请量为: 62.21448273460106
agent 4 在 53999 次迭代中的资源申请量为: 39.55143641630589
agent 5 在 53999 次迭代中的资源申请量为: 59.18904590023989
agent 6 在 53999 次迭代中的资源申请量为: 57.05973547603986
agent 7 在 53999 次迭代中的资源申请量为: 67.15435664802436
agent 8 在 53999 次迭代中的资源申请量为: 48.1743160725415
agent 9 在 53999 次迭代中的资源申请量为: 67.10964672927383
agent 10 在 53999 次迭代中的资源申请量为: 55.5898521809298
agent 11 在 53999 次迭代中的资源申请量为: 67.16111501592275
agent 12 在 53999 次迭代中的资源申请量为: 40.225029452920126
agent 13 在 53999 次迭代中的资源申请量为: 51.49257512903627
agent 14 在 53999 次迭代中的资源申请量为: 42.99014453177624
agent 15 在 53999 次迭代中的资源申请量为: 44.337709643029484
agent 16 在 53999 次迭代中的资源申请量为: 51.26398548166408
agent 17 在 53999 次迭代中的资源申请量为: 51.88024374435723
agent 18 在 53999 次迭代中的资源申请量为: 48.38585648075609
agent 19 在 53999 次迭代中的资源申请量为: 46.649610570897806
agent 20 在 53999 次迭代中的资源申请量为: 52.412919091258004
agent 21 在 53999 次迭代中的资源申请量为: 7

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.968747115850675
agent 2 在 55999 次迭代中的资源申请量为: 57.37517582550517
agent 3 在 55999 次迭代中的资源申请量为: 62.21448273460106
agent 4 在 55999 次迭代中的资源申请量为: 39.55143641630589
agent 5 在 55999 次迭代中的资源申请量为: 59.18904590023989
agent 6 在 55999 次迭代中的资源申请量为: 57.05973547603986
agent 7 在 55999 次迭代中的资源申请量为: 67.15435664802436
agent 8 在 55999 次迭代中的资源申请量为: 48.1743160725415
agent 9 在 55999 次迭代中的资源申请量为: 67.10964672927383
agent 10 在 55999 次迭代中的资源申请量为: 55.5898521809298
agent 11 在 55999 次迭代中的资源申请量为: 67.16111501592275
agent 12 在 55999 次迭代中的资源申请量为: 40.225029452920126
agent 13 在 55999 次迭代中的资源申请量为: 51.49257512903627
agent 14 在 55999 次迭代中的资源申请量为: 42.99014453177624
agent 15 在 55999 次迭代中的资源申请量为: 44.337709643029484
agent 16 在 55999 次迭代中的资源申请量为: 51.26398548166408
agent 17 在 55999 次迭代中的资源申请量为: 51.88024374435723
agent 18 在 55999 次迭代中的资源申请量为: 48.38585648075609
agent 19 在 55999 次迭代中的资源申请量为: 46.649610570897806
agent 20 在 55999 次迭代中的资源申请量为: 52.412919091258004
agent 21 在 55999 次迭代中的资源申请量为: 7

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.968747115850675
agent 2 在 57999 次迭代中的资源申请量为: 57.37517582550517
agent 3 在 57999 次迭代中的资源申请量为: 62.21448273460106
agent 4 在 57999 次迭代中的资源申请量为: 39.55143641630589
agent 5 在 57999 次迭代中的资源申请量为: 59.18904590023989
agent 6 在 57999 次迭代中的资源申请量为: 57.05973547603986
agent 7 在 57999 次迭代中的资源申请量为: 67.15435664802436
agent 8 在 57999 次迭代中的资源申请量为: 48.1743160725415
agent 9 在 57999 次迭代中的资源申请量为: 67.10964672927383
agent 10 在 57999 次迭代中的资源申请量为: 55.5898521809298
agent 11 在 57999 次迭代中的资源申请量为: 67.16111501592275
agent 12 在 57999 次迭代中的资源申请量为: 40.225029452920126
agent 13 在 57999 次迭代中的资源申请量为: 51.49257512903627
agent 14 在 57999 次迭代中的资源申请量为: 42.99014453177624
agent 15 在 57999 次迭代中的资源申请量为: 44.337709643029484
agent 16 在 57999 次迭代中的资源申请量为: 51.26398548166408
agent 17 在 57999 次迭代中的资源申请量为: 51.88024374435723
agent 18 在 57999 次迭代中的资源申请量为: 48.38585648075609
agent 19 在 57999 次迭代中的资源申请量为: 46.649610570897806
agent 20 在 57999 次迭代中的资源申请量为: 52.412919091258004
agent 21 在 57999 次迭代中的资源申请量为: 7

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.968747115850675
agent 2 在 59999 次迭代中的资源申请量为: 57.37517582550517
agent 3 在 59999 次迭代中的资源申请量为: 62.21448273460106
agent 4 在 59999 次迭代中的资源申请量为: 39.55143641630589
agent 5 在 59999 次迭代中的资源申请量为: 59.18904590023989
agent 6 在 59999 次迭代中的资源申请量为: 57.05973547603986
agent 7 在 59999 次迭代中的资源申请量为: 67.15435664802436
agent 8 在 59999 次迭代中的资源申请量为: 48.1743160725415
agent 9 在 59999 次迭代中的资源申请量为: 67.10964672927383
agent 10 在 59999 次迭代中的资源申请量为: 55.5898521809298
agent 11 在 59999 次迭代中的资源申请量为: 67.16111501592275
agent 12 在 59999 次迭代中的资源申请量为: 40.225029452920126
agent 13 在 59999 次迭代中的资源申请量为: 51.49257512903627
agent 14 在 59999 次迭代中的资源申请量为: 42.99014453177624
agent 15 在 59999 次迭代中的资源申请量为: 44.337709643029484
agent 16 在 59999 次迭代中的资源申请量为: 51.26398548166408
agent 17 在 59999 次迭代中的资源申请量为: 51.88024374435723
agent 18 在 59999 次迭代中的资源申请量为: 48.38585648075609
agent 19 在 59999 次迭代中的资源申请量为: 46.649610570897806
agent 20 在 59999 次迭代中的资源申请量为: 52.412919091258004
agent 21 在 59999 次迭代中的资源申请量为: 7

In [6]:
np.savetxt('./data/BREDA_TM_q=4_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/BREDA_TM_q=4_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/BREDA_TM_q=4_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/BREDA_TM_q=4_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# BREDA+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_robust2=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.1
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust2=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_robust2*100)):
    x_average_resource_quantity_robust2[i][0]=agent_resource_quantity_robust2[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    x_average_resource_quantity_robust2[i][0]=100    
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help)'''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust2_help=0\n    for j in range(0,15):\n        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:\n            q_robust2_help=q_robust2_help+1\n    q_robust2.append(q_robust2_help)'

In [10]:
def BREDA_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,x_average_resource_quantity_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************BREDA+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_robust2*100)):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]+iteration_update_robust2*((x_average_resource_quantity_robust2[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]-iteration_update_robust2*(((2*a_robust2[i]*agent_resource_quantity_robust2[i][k-1]-2*a_robust2[i]*b_robust2[i])/100)+(dual_variable_robust2[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_robust2*100)),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]
                
        #TM鲁棒聚合 每个用户聚合自身以及邻居的动态一致平均值  诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_robust2*100)):
            sort_list_robust2=[]
            sum_x_robust2=0
            for j in range (0,15):
                sort_list_robust2.append(x_average_resource_quantity_robust2[neighbor_robust2[i,j]][k-1])
            sort_list_robust2.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust2)):
                del sort_list_robust2[14-j]
            for j in range(0,int(q_robust2)):
                del sort_list_robust2[0]     
            for j in range(0,len(sort_list_robust2)):
                sum_x_robust2=sum_x_robust2+sort_list_robust2[j]            
            x_average_resource_quantity_robust2[i][k]=(1/(16-2*q_robust2))*(x_average_resource_quantity_robust2[i][k-1]+sum_x_robust2)+(agent_resource_quantity_robust2[i][k]-agent_resource_quantity_robust2[i][k-1])      
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_robust2*100)),100):
            x_average_resource_quantity_robust2[i][k]=x_average_resource_quantity_robust2[i][k-1]           
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_robust2[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        #iteration_update_robust2=1/(pow(k+1,0.5)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
BREDA_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,x_average_resource_quantity_robust2)

****************************BREDA+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 54.434924862561545
agent 2 在 3999 次迭代中的资源申请量为: 54.339467326359824
agent 3 在 3999 次迭代中的资源申请量为: 58.89803172093042
agent 4 在 3999 次迭代中的资源申请量为: 37.27442726581167
agent 5 在 3999 次迭代中的资源申请量为: 55.80335340837435
agent 6 在 3999 次迭代中的资源申请量为: 53.88090344568706
agent 7 在 3999 次迭代中的资源申请量为: 63.42231180828435
agent 8 在 3999 次迭代中的资源申请量为: 45.6036907026303
agent 9 在 3999 次迭代中的资源申请量为: 63.49860935038451
agent 10 在 3999 次迭代中的资源申请量为: 52.33471648350243
agent 11 在 3999 次迭代中的资源申请量为: 63.0530751881501
agent 12 在 3999 次迭代中的资源申请量为: 37.981817214738385
agent 13 在 3999 次迭代中的资源申请量为: 48.802529337149785
agent 14 在 3999 次迭代中的资源申请量为: 40.717234640348394
agent 15 在 3999 次迭代中的资源申请量为: 41.829345651004616
agent 16 在 3999 次迭代中的资源申请量为: 48.15117770114406
agent 17 在 3999 次迭代中的资源申请量为: 48.97246104701702
agent 18 在 3999 次迭代中的资源申请量为: 45.84411615191248
agent 19 在 3999 次迭代中的资源申请量为: 44.23654075483652
agent 20 在 3999 次迭代中的资源申请量为: 49.57501277179994
agent 21 在 3999 次迭代中的资源申请量为: 67.20098926381928
agen

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 54.41535033154483
agent 2 在 5999 次迭代中的资源申请量为: 54.32098997396929
agent 3 在 5999 次迭代中的资源申请量为: 58.88274589010297
agent 4 在 5999 次迭代中的资源申请量为: 37.24054670157575
agent 5 在 5999 次迭代中的资源申请量为: 55.785033958885506
agent 6 在 5999 次迭代中的资源申请量为: 53.86155864061305
agent 7 在 5999 次迭代中的资源申请量为: 63.409504753779615
agent 8 在 5999 次迭代中的资源申请量为: 45.57849404187407
agent 9 在 5999 次迭代中的资源申请量为: 63.48575724823945
agent 10 在 5999 次迭代中的资源申请量为: 52.314570402247874
agent 11 在 5999 次迭代中的资源申请量为: 63.03913504157061
agent 12 在 5999 次迭代中的资源申请量为: 37.948311981758344
agent 13 在 5999 次迭代中的资源申请量为: 48.78122191638084
agent 14 在 5999 次迭代中的资源申请量为: 40.686930380529496
agent 15 在 5999 次迭代中的资源申请量为: 41.80003408497776
agent 16 在 5999 次迭代中的资源申请量为: 48.127346678700086
agent 17 在 5999 次迭代中的资源申请量为: 48.94954002403403
agent 18 在 5999 次迭代中的资源申请量为: 45.81930713106041
agent 19 在 5999 次迭代中的资源申请量为: 44.21100511347646
agent 20 在 5999 次迭代中的资源申请量为: 49.55280127426049
agent 21 在 5999 次迭代中的资源申请量为: 67.1900707136542
age

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 54.407679439633014
agent 2 在 7999 次迭代中的资源申请量为: 54.3133895616635
agent 3 在 7999 次迭代中的资源申请量为: 58.87518876842356
agent 4 在 7999 次迭代中的资源申请量为: 37.2327352661229
agent 5 在 7999 次迭代中的资源申请量为: 55.77740246218703
agent 6 在 7999 次迭代中的资源申请量为: 53.85392517113596
agent 7 在 7999 次迭代中的资源申请量为: 63.401943238749354
agent 8 在 7999 次迭代中的资源申请量为: 45.57079647495423
agent 9 在 7999 次迭代中的资源申请量为: 63.47821214626749
agent 10 在 7999 次迭代中的资源申请量为: 52.3069163933725
agent 11 在 7999 次迭代中的资源申请量为: 63.03150173748157
agent 12 在 7999 次迭代中的资源申请量为: 37.94051019433072
agent 13 在 7999 次迭代中的资源申请量为: 48.77359238488523
agent 14 在 7999 次迭代中的资源申请量为: 40.6791704977201
agent 15 在 7999 次迭代中的资源申请量为: 41.79226988819733
agent 16 在 7999 次迭代中的资源申请量为: 48.11962752248935
agent 17 在 7999 次迭代中的资源申请量为: 48.941857578085255
agent 18 在 7999 次迭代中的资源申请量为: 45.811619272759344
agent 19 在 7999 次迭代中的资源申请量为: 44.20331481665388
agent 20 在 7999 次迭代中的资源申请量为: 49.54513866817292
agent 21 在 7999 次迭代中的资源申请量为: 67.18250491783368
agent 22

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 54.407223397609904
agent 2 在 9999 次迭代中的资源申请量为: 54.31293827504834
agent 3 在 9999 次迭代中的资源申请量为: 58.87472931053315
agent 4 在 9999 次迭代中的资源申请量为: 37.23232170471925
agent 5 在 9999 次迭代中的资源申请量为: 55.77694621422997
agent 6 在 9999 次迭代中的资源申请量为: 53.85347335580732
agent 7 在 9999 次迭代中的资源申请量为: 63.401474550962604
agent 8 在 9999 次迭代中的资源申请量为: 45.57036225156462
agent 9 在 9999 次迭代中的资源申请量为: 63.47774560855718
agent 10 在 9999 次迭代中的资源申请量为: 52.30646500201699
agent 11 在 9999 次迭代中的资源申请量为: 63.031028393862876
agent 12 在 9999 次迭代中的资源申请量为: 37.94009623256532
agent 13 在 9999 次迭代中的资源申请量为: 48.773150010813055
agent 14 在 9999 次迭代中的资源申请量为: 40.67874946773225
agent 15 在 9999 次迭代中的资源申请量为: 41.79184426057685
agent 16 在 9999 次迭代中的资源申请量为: 48.11918437175245
agent 17 在 9999 次迭代中的资源申请量为: 48.94141535205218
agent 18 在 9999 次迭代中的资源申请量为: 45.811184678760206
agent 19 在 9999 次迭代中的资源申请量为: 44.20288300527095
agent 20 在 9999 次迭代中的资源申请量为: 49.54469598790204
agent 21 在 9999 次迭代中的资源申请量为: 67.18203053092866
age

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 54.40721735435268
agent 2 在 51999 次迭代中的资源申请量为: 54.31293254379975
agent 3 在 51999 次迭代中的资源申请量为: 58.87472275167025
agent 4 在 51999 次迭代中的资源申请量为: 37.23231942875985
agent 5 在 51999 次迭代中的资源申请量为: 55.77694008481187
agent 6 在 51999 次迭代中的资源申请量为: 53.85346762525924
agent 7 在 51999 次迭代中的资源申请量为: 63.40146715598526
agent 8 在 51999 次迭代中的资源申请量为: 45.57035814582754
agent 9 在 51999 次迭代中的资源申请量为: 63.47773838199929
agent 10 在 51999 次迭代中的资源申请量为: 52.306459355592466
agent 11 在 51999 次迭代中的资源申请量为: 63.03102070262075
agent 12 在 51999 次迭代中的资源申请量为: 37.940093904368325
agent 13 在 51999 次迭代中的资源申请量为: 48.7731451034348
agent 14 在 51999 次迭代中的资源申请量为: 40.67874653939491
agent 15 在 51999 次迭代中的资源申请量为: 41.79184095964068
agent 16 在 51999 次迭代中的资源申请量为: 48.119179536710334
agent 17 在 51999 次迭代中的资源申请量为: 48.9414105445878
agent 18 在 51999 次迭代中的资源申请量为: 45.81118052750912
agent 19 在 51999 次迭代中的资源申请量为: 44.20287909037209
agent 20 在 51999 次迭代中的资源申请量为: 49.54469111325761
agent 21 在 51999 次迭代中的资源申请量为: 67.

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 54.40721735435268
agent 2 在 53999 次迭代中的资源申请量为: 54.31293254379975
agent 3 在 53999 次迭代中的资源申请量为: 58.87472275167025
agent 4 在 53999 次迭代中的资源申请量为: 37.23231942875985
agent 5 在 53999 次迭代中的资源申请量为: 55.77694008481187
agent 6 在 53999 次迭代中的资源申请量为: 53.85346762525924
agent 7 在 53999 次迭代中的资源申请量为: 63.40146715598526
agent 8 在 53999 次迭代中的资源申请量为: 45.57035814582754
agent 9 在 53999 次迭代中的资源申请量为: 63.47773838199929
agent 10 在 53999 次迭代中的资源申请量为: 52.306459355592466
agent 11 在 53999 次迭代中的资源申请量为: 63.03102070262075
agent 12 在 53999 次迭代中的资源申请量为: 37.940093904368325
agent 13 在 53999 次迭代中的资源申请量为: 48.7731451034348
agent 14 在 53999 次迭代中的资源申请量为: 40.67874653939491
agent 15 在 53999 次迭代中的资源申请量为: 41.79184095964068
agent 16 在 53999 次迭代中的资源申请量为: 48.119179536710334
agent 17 在 53999 次迭代中的资源申请量为: 48.9414105445878
agent 18 在 53999 次迭代中的资源申请量为: 45.81118052750912
agent 19 在 53999 次迭代中的资源申请量为: 44.20287909037209
agent 20 在 53999 次迭代中的资源申请量为: 49.54469111325761
agent 21 在 53999 次迭代中的资源申请量为: 67.

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 54.40721735435268
agent 2 在 55999 次迭代中的资源申请量为: 54.31293254379975
agent 3 在 55999 次迭代中的资源申请量为: 58.87472275167025
agent 4 在 55999 次迭代中的资源申请量为: 37.23231942875985
agent 5 在 55999 次迭代中的资源申请量为: 55.77694008481187
agent 6 在 55999 次迭代中的资源申请量为: 53.85346762525924
agent 7 在 55999 次迭代中的资源申请量为: 63.40146715598526
agent 8 在 55999 次迭代中的资源申请量为: 45.57035814582754
agent 9 在 55999 次迭代中的资源申请量为: 63.47773838199929
agent 10 在 55999 次迭代中的资源申请量为: 52.306459355592466
agent 11 在 55999 次迭代中的资源申请量为: 63.03102070262075
agent 12 在 55999 次迭代中的资源申请量为: 37.940093904368325
agent 13 在 55999 次迭代中的资源申请量为: 48.7731451034348
agent 14 在 55999 次迭代中的资源申请量为: 40.67874653939491
agent 15 在 55999 次迭代中的资源申请量为: 41.79184095964068
agent 16 在 55999 次迭代中的资源申请量为: 48.119179536710334
agent 17 在 55999 次迭代中的资源申请量为: 48.9414105445878
agent 18 在 55999 次迭代中的资源申请量为: 45.81118052750912
agent 19 在 55999 次迭代中的资源申请量为: 44.20287909037209
agent 20 在 55999 次迭代中的资源申请量为: 49.54469111325761
agent 21 在 55999 次迭代中的资源申请量为: 67.

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 54.40721735435268
agent 2 在 57999 次迭代中的资源申请量为: 54.31293254379975
agent 3 在 57999 次迭代中的资源申请量为: 58.87472275167025
agent 4 在 57999 次迭代中的资源申请量为: 37.23231942875985
agent 5 在 57999 次迭代中的资源申请量为: 55.77694008481187
agent 6 在 57999 次迭代中的资源申请量为: 53.85346762525924
agent 7 在 57999 次迭代中的资源申请量为: 63.40146715598526
agent 8 在 57999 次迭代中的资源申请量为: 45.57035814582754
agent 9 在 57999 次迭代中的资源申请量为: 63.47773838199929
agent 10 在 57999 次迭代中的资源申请量为: 52.306459355592466
agent 11 在 57999 次迭代中的资源申请量为: 63.03102070262075
agent 12 在 57999 次迭代中的资源申请量为: 37.940093904368325
agent 13 在 57999 次迭代中的资源申请量为: 48.7731451034348
agent 14 在 57999 次迭代中的资源申请量为: 40.67874653939491
agent 15 在 57999 次迭代中的资源申请量为: 41.79184095964068
agent 16 在 57999 次迭代中的资源申请量为: 48.119179536710334
agent 17 在 57999 次迭代中的资源申请量为: 48.9414105445878
agent 18 在 57999 次迭代中的资源申请量为: 45.81118052750912
agent 19 在 57999 次迭代中的资源申请量为: 44.20287909037209
agent 20 在 57999 次迭代中的资源申请量为: 49.54469111325761
agent 21 在 57999 次迭代中的资源申请量为: 67.

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 54.40721735435268
agent 2 在 59999 次迭代中的资源申请量为: 54.31293254379975
agent 3 在 59999 次迭代中的资源申请量为: 58.87472275167025
agent 4 在 59999 次迭代中的资源申请量为: 37.23231942875985
agent 5 在 59999 次迭代中的资源申请量为: 55.77694008481187
agent 6 在 59999 次迭代中的资源申请量为: 53.85346762525924
agent 7 在 59999 次迭代中的资源申请量为: 63.40146715598526
agent 8 在 59999 次迭代中的资源申请量为: 45.57035814582754
agent 9 在 59999 次迭代中的资源申请量为: 63.47773838199929
agent 10 在 59999 次迭代中的资源申请量为: 52.306459355592466
agent 11 在 59999 次迭代中的资源申请量为: 63.03102070262075
agent 12 在 59999 次迭代中的资源申请量为: 37.940093904368325
agent 13 在 59999 次迭代中的资源申请量为: 48.7731451034348
agent 14 在 59999 次迭代中的资源申请量为: 40.67874653939491
agent 15 在 59999 次迭代中的资源申请量为: 41.79184095964068
agent 16 在 59999 次迭代中的资源申请量为: 48.119179536710334
agent 17 在 59999 次迭代中的资源申请量为: 48.9414105445878
agent 18 在 59999 次迭代中的资源申请量为: 45.81118052750912
agent 19 在 59999 次迭代中的资源申请量为: 44.20287909037209
agent 20 在 59999 次迭代中的资源申请量为: 49.54469111325761
agent 21 在 59999 次迭代中的资源申请量为: 67.

In [12]:
np.savetxt('./data/BREDA_TM_q=4_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/BREDA_TM_q=4_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/BREDA_TM_q=4_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/BREDA_TM_q=4_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量